# Load model

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image


def load_image(path):
    return np.array(Image.open(path))


def plot_sample(lr, sr):
    plt.figure(figsize=(20, 10))

    images = [lr, sr]
    titles = ['LR', f'SR (x{sr.shape[0] // lr.shape[0]})']

    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(1, 2, i+1)
        plt.imshow(img)
        plt.title(title)
        plt.xticks([])
        plt.yticks([])

In [7]:
#from model.srgan import generator

#model = generator()
#model.load_weights('weights/srgan/gan_generator.h5')


path = "Dataset/test_LR/Scarlett Johansson/scarlett Johansson01.jpg"
lr = load_image(path)
#sr = resolve_single(model, lr)

plot_sample(lr, lr)

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/test_LR/Scarlett Johansson/scarlett Johansson01.jpg'